In [1]:
import numpy as np
import pandas as pd
import xarray as xr

from findlay2025a import core, hypnograms, sharp_waves
from findlay2025a.constants import Experiments as Exps


In [2]:
nb = core.get_project("seahorse")
s3 = core.get_project("shared")


def aggregate_sgfg(experiments=[Exps.NOD, Exps.COW, Exps.CTN]):
    df = pd.DataFrame()
    for subject, experiment in core.yield_subject_name_experiment_pairs(experiments):
        sg = xr.load_dataarray(
            nb.get_experiment_subject_file(
                experiment, subject, "AeryJones_slow_gamma.nc"
            )
        )
        fg = xr.load_dataarray(
            nb.get_experiment_subject_file(
                experiment, subject, "AeryJones_fast_gamma.nc"
            )
        )
        ds = xr.Dataset({"Slow.Gamma": sg, "Fast.Gamma": fg})
        ds["Slow.Fast.Gamma.Ratio"] = ds["Slow.Gamma"] / ds["Fast.Gamma"]

        # Rename coordinates for cleaner plots.
        ds = ds.rename({"roi": "ROI", "time": "Time"})

        # Add state as a coordinate on the time dimension of sgfg
        hgs = hypnograms.load_statistical_condition_hypnograms(
            experiment, subject, include_full_conservative=True
        )
        hg = hgs.pop("full_conservative").drop_states(core.ARTIFACT_STATES + ["NoData"])
        states = hg.get_states(ds["Time"].values)
        ds = ds.assign_coords({"State": ("Time", states)})

        # Add number of SPWs as a coordinate on the time dimension of sgfg
        spws = sharp_waves.read_spws(subject, experiment)
        spws = spws.iloc[hg.covers_time(spws["pk_time"])]
        nspw = np.zeros_like(ds["Time"].values)
        nt = ds["Time"].values.size
        for i in np.searchsorted(ds["Time"].values, spws["pk_time"].values):
            nspw[min(i, nt - 1)] += 1
        ds = ds.assign_coords({"SPW.Count": ("Time", nspw)})

        # Convert to dataframe
        _df = ds.to_dataframe().reset_index()[
            [
                "Time",
                "Slow.Gamma",
                "Fast.Gamma",
                "Slow.Fast.Gamma.Ratio",
                "SPW.Count",
                "State",
                "ROI",
            ]
        ]
        _df["Subject"] = subject
        _df["Experiment"] = experiment

        # Keep only Wake and NREM states
        _df = _df[_df["State"].isin(["Wake", "NREM"])]

        df = pd.concat([df, _df], axis=0, ignore_index=True)
    return df

In [3]:
df = aggregate_sgfg()
experiment_display_names = {
    Exps.NOD: "Novelty",
    Exps.COW: "Locomotion",
    Exps.CTN: "Dual",
}
df["Experiment"] = df["Experiment"].map(experiment_display_names)

has_spw = df["SPW.Count"] > 0
is_wake = df["State"] == "Wake"
is_nrem = df["State"] == "NREM"

df.loc[is_wake & has_spw, "Epoch.Type"] = "SPW.Wake"
df.loc[is_wake & ~has_spw, "Epoch.Type"] = "No.SPW.Wake"
df.loc[is_nrem & has_spw, "Epoch.Type"] = "SPW.NREM"
df.loc[is_nrem & ~has_spw, "Epoch.Type"] = "No.SPW.NREM"
df.to_parquet(nb.get_project_file("AeryJones_sgfg.pqt"))

<frozen importlib._bootstrap>:488: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 16 from C header, got 96 from PyObject


In [5]:
sgfg_medians_by_epoch_type = (
    df.groupby(["Subject", "Experiment", "ROI", "Epoch.Type"])[
        ["Slow.Fast.Gamma.Ratio", "Slow.Gamma", "Fast.Gamma"]
    ]
    .median()
    .reset_index()
)
sgfg_medians_by_epoch_type.to_parquet(
    nb.get_project_file(
        "AeryJones_sgfg_medians_by_epoch_type.pqt"
    )
)